___
<img style="float: right; margin: 15px 15px 15px 15px;" src="https://cdn.brandfetch.io/idjblR01Uf/w/400/h/400/theme/dark/icon.jpeg?c=1dxbfHSJFAPEGdCLU4o5B" width="180px" height="180px" />

# <font color= #5aafc1> **PROYECTO FINAL** </font>
#### <font color= #2E9AFE> `Red Aeroportuaria Internacional`</font>
- <Strong> Sofía Maldonado, Viviana Toledo, César Adrián Santos </Strong>
- <Strong> Fecha </Strong>: 17/07/2025

___

In [36]:
# Actualizar 
import importlib
import queries 
importlib.reload(queries)
from queries import DB_Queries

# Establecer conexión
db = DB_Queries('neo5j')

Successfully Connected to Neo4j in neo5j database!
Successfully Connected to GraphDataScience in neo5j database!


# <font color= #5aafc1> **Desarrollo** </font>

Añadir atributos de Continente a cada nodo:

In [ ]:
query1 = """MATCH (a:Airport) 
WHERE a.country = 'Burma'
SET a.country = 'Myanmar'
"""

query2 = """MATCH (a:Airport) 
WHERE a.country = 'Congo (Kinshasa)'
SET a.country = 'Democratic Republic of the Congo'
"""

query3 = """MATCH (a:Airport) 
WHERE a.country = 'Congo (Brazzaville)'
SET a.country = 'Republic of the Congo'
"""

query4 = f"""LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ch0fas/MG_PROYECTO2/main/data/countries.csv' AS row
MATCH (a:Airport)
WHERE a.country = row.Country
SET a.continent = row.Continent
"""

db.custom_query(query1)
db.custom_query(query2)
db.custom_query(query3)
db.custom_query(query4)

Crear subgrafos:

In [6]:
db.create_subgraph('Airport', 'TO', 'NATURAL', 'distance')
db.create_subgraph('Airport', 'TO', 'REVERSE', 'distance')
db.create_subgraph('Airport', 'TO', 'UNDIRECTED', 'distance')

Successfully created Directed Subgraph with Airport nodes and TO relationships!
Successfully created Reversed Subgraph with Airport nodes and TO relationships!
Successfully created Undirected Subgraph with Airport nodes and TO relationships!


Centrality Algorithms:

In [6]:
db.degrees('Airport')
db.page_rank('Airport')
db.betwenness('Airport')
db.closeness('Airport')

Degree Attribute added Successfully!
In_Degree Attribute added Successfully!
Out_Degree Attribute added Successfully!
Page Rank Attribute added Successfully!
Betweenness Attribute added Successfully!
Closeness Attribute added Successfully!


#### <font color= #5aafc1> **4.2 Exploratory Data Analysis** </font>

Graph Schema:

Node Count:

In [ ]:
# 4.2.2
db.count_nodes('Airport')

,Airports
0,3214


Node Attributes:

In [10]:
db.node_attributes()

,nodeType,nodeLabels,propertyName,propertyTypes,mandatory
0,:`Airport`,[Airport],Id,[Long],True
1,:`Airport`,[Airport],latitude,[Double],True
2,:`Airport`,[Airport],country,[String],True
3,:`Airport`,[Airport],longitude,[Double],True
4,:`Airport`,[Airport],Label,[String],True
5,:`Airport`,[Airport],continent,[String],True


Relationship Count:

In [11]:
db.count_relationships('TO')

,TO_Count
0,36907


Relationship Attributes:

In [12]:
db.rel_attributes()

,relType,propertyName,propertyTypes,mandatory
0,:`TO`,distance,[Double],True


#### <font color= #5aafc1> **4.3** </font>

In [4]:
# Le voy a agregar atributos de latitud y longitud en integer para poder meterlo a Gephi 

query4_3 = f"""
    MATCH (n)
    SET n.latInt = toInteger(n.latitude)
    SET n.longInt = toInteger(n.longitude)
"""

db.custom_query(query4_3)

""


#### <font color= #5aafc1> **4.4** </font>

#### <font color= #5aafc1> **4.5 Elige el nodo de la pregunta 4.4 como origen y uno de los posibles aeropuertos alcanzables de la misma pregunta como destino, muestra la ruta completa del origen al destino en un mapa.** </font>

In [35]:
query4_5 = f""" 
    MATCH(a:Airport {{Label:'Villahermosa'}})-[r1]->(b)-[r2]->(c)-[r3]->(m:Airport {{Label:'Delhi'}})
    WHERE r3.distance IS NOT NULL
    RETURN a.Label,r1.distance AS Dist1,b.Label,r2.distance AS Dist2,c.Label,r3.distance AS Dist3,m.Label
    ORDER BY Dist1+Dist2+Dist3 DESC
    LIMIT 10;
"""

db.custom_query(query4_5)

,a.Label,Dist1,b.Label,Dist2,c.Label,Dist3,m.Label
0,Villahermosa,1358.755582,Houston,10681.305092,Tokyo,5915.459645,Delhi
1,Villahermosa,1358.755582,Houston,11358.558584,Seoul,4657.687448,Delhi
2,Villahermosa,1358.755582,Houston,12947.031367,Doha,2552.069255,Delhi
3,Villahermosa,1358.755582,Houston,11569.256010,Beijing,3813.207294,Delhi
4,Villahermosa,1358.755582,Houston,13139.523542,Dubai,2186.045810,Delhi
5,Villahermosa,678.579085,Mexico City,9209.229377,Paris,6572.730383,Delhi
6,Villahermosa,678.579085,Mexico City,9855.892994,Munich,5906.062233,Delhi
7,Villahermosa,678.579085,Mexico City,9562.676617,Frankfurt,6127.480196,Delhi
8,Villahermosa,678.579085,Mexico City,8913.836258,London,6738.831316,Delhi
9,Villahermosa,678.579085,Mexico City,9216.602463,Amsterdam,6370.585848,Delhi


#### <font color= #5aafc1> **4.6** </font>

#### <font color= #5aafc1> **4.7** </font>

#### <font color= #5aafc1> **4.8. Imagina que vives en el país que elegiste en el punto 4.7 (Colombia) y tienes la posibilidad de viajar a cualquier parte del mundo desde su aeropuerto más importante.** </font>

Primero encontraremos el aeropuerto más importante de Colombia en base a la medida de centralidad _Page Rank_, que evalúa las conexiones que salen del nodo y les asigna una relevancia.

In [14]:
query1 = """MATCH (a:Airport)
WHERE a.country = 'Colombia'
RETURN a.Label AS Airport, a.country AS Country, a.Page_Rank AS Page_Rank
ORDER BY Page_Rank DESC
LIMIT 1
"""

db.custom_query(query1)

,Airport,Country,Page_Rank
0,Bogota,Colombia,9.420864


El aeropuerto más importante de Colombia es el de Bogotá, que se ubica en la capital.

#### <font color= #5aafc1> &nbsp; &nbsp; **4.8.1. ¿Cuántos kilómetros se requieren para visitar las 7 maravillas del mundo?** </font>
#### <font color= #5aafc1> &nbsp; &nbsp; **4.8.2. ¿Cuál sería la mejor ruta para visitarlas todas, en qué orden y por qué?** </font>

Decidimos trabajar con las siguientes 7 maravillas del mundo modernas:

- Giza Pyramids: Cairo
- Great Wall of China: Beijing
- Colosseum: Rome
- Chichen Itzá: Mérida
- Machu Picchu: Cuzco
- Taj Mahal: Lucknow
- Christ the Redeemer: Rio de Janeiro

Para dar respuesta a las preguntas, se analizaron todas las combinaciones posibles desde Bogotá hacia los destinos mencionados. Entonces, a partir de Bogotá, la mejor ruta para recorrer las 7 maravillas del mundo moderno es:

In [12]:
db.seven_wonders_best_route('Airport','Label','Bogota','distance')

,order,distance
0,"(Bogota, Merida, Cuzco, Rio De Janeiro, Rome, Cairo, Lucknow, Beijing)",29588.070071


In [37]:
db.temp_dijkstra_directed('Airport', 'Label', 'Bogota', 'Merida', 'distance')

,Source,Target,Bridges,Distance
0,Bogota,Merida,"[Bogota, Panama City, San Pedro Sula, Belize City, Merida]",2564.717484
1,Bogota,Merida,"[Bogota, Caracas, Merida]",1536.516855


Esto requiere alrededor de 29,588 km.

#### <font color= #5aafc1> &nbsp; &nbsp; **4.8.3. Aeropuertos visitables desde el Aeropuerto más importante de Colombia en menos de 20,000km** </font>

Crearemos un atributo que nos permita evaluar todas las rutas posibles desde Bogotá:

In [58]:
db.delta_pathing('Airport', 'Label', 'Bogota', 'distance')

Directed Delta Pathing for Bogota already exists. Creation skipped.


,Source,Bridges,Partial_Costs,Total_Cost,Target
0,Bogota,"[Goroka, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]","[0.0, 755.8972047746067, 5606.6853979173375, 14508.500729366064, 17901.63346808019, 18326.70207030612]",18326.702070,Goroka
1,Bogota,"[Madang, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]","[0.0, 755.8972047746067, 5606.6853979173375, 14508.500729366064, 17901.63346808019, 18398.968325213587]",18398.968325,Madang
2,Bogota,"[Mount Hagen, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]","[0.0, 755.8972047746067, 5606.6853979173375, 14508.500729366064, 17901.63346808019, 18417.507717458266]",18417.507717,Mount Hagen


Undirected Delta Pathing for Bogota already exists. Creation skipped.


,Source,Bridges,Partial_Costs,Total_Cost,Target
0,Bogota,"[Goroka, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]","[0.0, 755.8972047746067, 5606.6853979173375, 14508.500729366064, 17901.63346808019, 18326.70207030612]",18326.702070,Goroka
1,Bogota,"[Madang, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]","[0.0, 755.8972047746067, 5606.6853979173375, 14508.500729366064, 17901.63346808019, 18398.968325213587]",18398.968325,Madang
2,Bogota,"[Mount Hagen, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]","[0.0, 755.8972047746067, 5606.6853979173375, 14508.500729366064, 17901.63346808019, 18417.507717458266]",18417.507717,Mount Hagen


Los aeropuertos y rutas visitables que sumen menos de 20,000 km:

In [11]:
query2 = """MATCH (source:Airport) -[r:PATH_Bogota_Directed]-> (target:Airport)
WHERE r.totalCost <= 20_000 
RETURN COUNT(DISTINCT target.Label) AS Distinct_Visitable_Airports
"""
 
db.custom_query(query2)

,Distinct_Visitable_Airports
0,3001


In [55]:
query3 = """MATCH (source:Airport) -[r:PATH_Bogota_Directed]-> (target:Airport)
MATCH (v:Airport)
WHERE r.totalCost <= 20_000 AND id(v) IN r.nodeIds
RETURN source.Label AS Source, target.Label AS Target, COLLECT(v.Label) AS Bridges, 
target.country AS Target_Country, r.totalCost AS Kilometers
LIMIT 20
"""
 
db.custom_query(query3)

,Source,Target,Bridges,Target_Country,Kilometers
0,Bogota,Goroka,"[Goroka, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]",Papua New Guinea,18326.702070
1,Bogota,Madang,"[Madang, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]",Papua New Guinea,18398.968325
2,Bogota,Mount Hagen,"[Mount Hagen, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]",Papua New Guinea,18417.507717
3,Bogota,Nadzab,"[Nadzab, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]",Papua New Guinea,18226.120224
4,Bogota,Port Moresby,"[Port Moresby, Panama City, Nandi, Bogota, Los Angeles]",Papua New Guinea,17901.633468
5,Bogota,Wewak,"[Port Moresby, Wewak, Panama City, Nandi, Bogota, Los Angeles]",Papua New Guinea,18662.934623
6,Bogota,Narssarssuaq,"[Narssarssuaq, Godthaab, Keflavik, Santo Domingo, Bogota, Boston]",Greenland,10016.244656
7,Bogota,Godthaab,"[Godthaab, Keflavik, Santo Domingo, Bogota, Boston]",Greenland,9552.032638
8,Bogota,Sondrestrom,"[Godthaab, Sondrestrom, Keflavik, Santo Domingo, Bogota, Boston]",Greenland,9869.219679
9,Bogota,Thule,"[Godthaab, Thule, Keflavik, Santo Domingo, Bogota, Boston, Ilulissat, Qaanaaq, Upernavik]",Greenland,11300.657141


Algunos de los destinos visitables desde Bogotá a menos de 20,000 km de distancia son Papua New Guinea, Greenland, Iceland y Canada. Las rutas para llegar a cada destino están especificadas en la tabla de arriba, mostrando los países a recorrer.

## <font color= #5aafc1> **Tours** </font>

## <font color= #EEF4ED> **Tour 1: The Grand Tour (Europa)** </font>

In [2]:
# Subgrafo específico de aeropuertos en Europa
db.create_subgraph_continent('Airport','TO','Europe')

Successfully created Airport_Europe Subgraph with Airport nodes and TO relationships!


In [3]:
# 1
db.custom_query(f"""
    CALL gds.degree.write('Airport_Europe', {{writeProperty: 'Out_Degree_Europe'}} )
    YIELD centralityDistribution, nodePropertiesWritten
    RETURN centralityDistribution.min AS minimumScore, 
    centralityDistribution.mean AS meanScore, 
    nodePropertiesWritten;
""")

,minimumScore,meanScore,nodePropertiesWritten
0,0.0,19.43743,503


In [4]:
db.custom_query("""MATCH (a:Airport {continent:'Europe'})
                WHERE a.Out_Degree_Europe IS NOT null
                RETURN a.Label AS Name, a.Out_Degree_Europe AS ODE
                ORDER BY ODE DESC
                LIMIT 10;""")

,Name,ODE
0,London,145.0
1,Amsterdam,141.0
2,Munich,133.0
3,Barcelona,131.0
4,Dublin,128.0
5,Palma de Mallorca,123.0
6,London,123.0
7,Frankfurt,120.0
8,Paris,113.0
9,Duesseldorf,109.0


In [5]:
# 2
db.custom_query(f"""
    CALL gds.pageRank.write(
        'Airport_Europe',
        {{writeProperty: 'Page_Rank_Europe'}} )
        YIELD nodePropertiesWritten, ranIterations;
""")

,nodePropertiesWritten,ranIterations
0,503,20


In [10]:
top_europe_countries = ['Germany','United Kingdom','France','Italy','Spain','Ukraine','Poland','Romania','Netherlands','Belgium']

for i in top_europe_countries:
    result = db.custom_query(f"""
    MATCH (a:Airport {{country: '{i}'}})
    WHERE a.Page_Rank_Europe IS NOT NULL
    RETURN a.Label AS Airport, a.Page_Rank_Europe AS PR
    ORDER BY PR DESC
    LIMIT 1;
    """)

    print(f"{i}: {result['Airport']}")


Germany: 0    Munich
Name: Airport, dtype: object
United Kingdom: 0    London
Name: Airport, dtype: object
France: 0    Paris
Name: Airport, dtype: object
Italy: 0    Rome
Name: Airport, dtype: object
Spain: 0    Barcelona
Name: Airport, dtype: object
Ukraine: 0    Kiev
Name: Airport, dtype: object
Poland: 0    Warsaw
Name: Airport, dtype: object
Romania: 0    Bucharest
Name: Airport, dtype: object
Netherlands: 0    Amsterdam
Name: Airport, dtype: object
Belgium: 0    Brussels
Name: Airport, dtype: object


In [35]:
# 3
cities = ['Munich','London','Paris','Rome','Barcelona','Kiev','Warsaw','Bucharest','Amsterdam','Brussels']

for i in cities:
    result = db.temp_dijkstra_directed('Airport','Label','Guadalajara',i,'distance')
    print(f"""
    Ruta GDL-{i}: {result["Distance"]} \n
    Middle Airports: {result['Middle']}
""")


    Ruta GDL-Munich: 0    10031.953936
Name: Distance, dtype: float64 

    Middle Airports: 0    [818, 1744, 196]
Name: Middle, dtype: object


    Ruta GDL-London: 0    3358.698645
1    9086.463951
2    9126.420797
3    9131.317161
4    9094.852864
5    9123.639416
Name: Distance, dtype: float64 

    Middle Airports: 0              [818, 1885, 109]
1    [818, 1744, 125, 288, 247]
2          [818, 1744, 89, 253]
3    [818, 1744, 125, 288, 254]
4              [818, 1744, 255]
5    [818, 1744, 125, 288, 279]
Name: Middle, dtype: object


    Ruta GDL-Paris: 0    9403.563969
1    9407.824968
Name: Distance, dtype: float64 

    Middle Airports: 0        [818, 1744, 628]
1    [818, 1744, 89, 629]
Name: Middle, dtype: object


    Ruta GDL-Rome: 0    10517.777807
1    10460.905875
Name: Distance, dtype: float64 

    Middle Airports: 0    [818, 1744, 89, 618, 705]
1             [818, 1809, 706]
Name: Middle, dtype: object


    Ruta GDL-Barcelona: 0    9746.264626
1    4296.911075
Name: 

Estas rutas son complejas y difíciles de tratar para una persona turista, por ende se propone la ruta de GDL a Londres via la CDMX, la cuál es apenas 300 km más larga pero es logísticamente más sencilla

In [44]:
db.custom_query(f"MATCH (a:Airport {{Label:'Guadalajara'}})-[r:TO]->(a2:Airport {{Label: 'Mexico City'}})-[r2:TO]->(a3:Airport {{Label: 'London'}}) RETURN r.distance + r2.distance, a3.Id")

,r.distance + r2.distance,a3.Id
0,9373.457896,507


De forma similar, para el retorno, se propone la siguiente ruta: Barcelona-GDL via Londres y CDMX

In [ ]:
db.custom_query(f"MATCH (a:Airport {{Label:'Barcelona'}})-[r:TO]->(a2:Airport {{Label: 'London'}})RETURN r.distance, a2.Id") # se va a tomar el aeropuerto con id=507

,r.distance,a2.Id
0,1192.097273,492
1,1110.296837,502
2,1148.999108,507
3,1186.942188,548


In [45]:
# 4
grand_1 = db.custom_query(f"""
    MATCH (a:Airport {{Id:507}})-[r:TO]->(a2:Airport {{Label:'Munich'}}) 
    RETURN r.distance AS Dist;
""") # En este caso, identifico a Londres con su Id específico ya que hay varios aeropuertos con el Label:"Londres". Este es el mismo aeropuerto en el que llegamos

grand_2 = db.custom_query(f"""
    MATCH (a:Airport {{Label:'Munich'}})-[r:TO]->(a2:Airport {{Label:'Paris'}}) 
    RETURN r.distance AS Dist;
""")

grand_3 = db.custom_query(f"""
    MATCH (a:Airport {{Label:'Paris'}})-[r:TO]->(a2:Airport {{Label:'Rome'}}) 
    RETURN r.distance AS Dist;
""")

grand_4 = db.custom_query(f"""
    MATCH (a:Airport {{Label:'Rome'}})-[r:TO]->(a2:Airport {{Label:'Barcelona'}}) 
    RETURN r.distance AS Dist;
""")

grand_f = db.temp_dijkstra_directed('Airport','Label','Barcelona','Guadalajara','distance')

print(f"""
    Grand Tour Report: \n
    Total Distance: {9373+grand_1['Dist']+grand_2['Dist']+grand_3['Dist']+grand_4['Dist']+(1186+9373)}
    Cost Per Leg: 
""")


    Grand Tour Report: 

    Total Distance: 0    23536.266188
1             NaN
Name: Dist, dtype: float64
    Cost Per Leg: 



In [39]:
db.temp_dijkstra_directed('Airport','Label','Barcelona','Guadalajara','distance')

,Source,Target,Middle,Distance
0,Barcelona,Guadalajara,"[570, 1854, 1809, 818]",9746.264626
1,Barcelona,Guadalajara,"[1329, 1340, 832, 818]",4296.911075


In [48]:
# 5
db.custom_query(f"""CALL gds.louvain.write(
            'Airport_Europe',
            {{writeProperty: 'Louvain_Europe'}} )
            YIELD communityCount, modularity, modularities;
            """)

,communityCount,modularity,modularities
0,8,0.267822,"[0.2541399296152414, 0.26615043547199513, 0.26782181796958543]"


In [49]:
db.custom_query(f"""
    MATCH (a:Airport {{continent:'Europe'}})
    RETURN a.Louvain_Europe AS LV, COLLECT(DISTINCT a.country) AS Countries
""")

,LV,Countries
0,1.0,[Iceland]
1,45.0,"[Iceland, Estonia, Finland, United Kingdom, Netherlands, Denmark, Faroe Islands, Norway, Poland, Sweden, Bulgaria, Cyprus, Azerbaijan, Ukraine, Belarus, Latvia, Lithuania, Armenia, Georgia]"
2,66.0,"[Belgium, Germany, United Kingdom, Netherlands, Spain, Albania, France, Greece, Italy, Czech Republic, Portugal, Switzerland, Montenegro]"
3,135.0,"[Belgium, Germany, Finland, United Kingdom, Netherlands, Norway, Poland, Sweden, Bulgaria, Croatia, Spain, France, Greece, Hungary, Italy, Czech Republic, Bosnia and Herzegovina, Romania, Moldova, Macedonia, Slovakia, Ukraine]"
4,128.0,"[Germany, United Kingdom, Guernsey, Jersey, Isle of Man, Ireland, Poland, Sweden, Spain, Cyprus, France, Malta, Portugal, Gibraltar, Lithuania]"
5,457.0,"[Germany, Luxembourg, Spain, Bulgaria, Croatia, Greece, Italy, Slovenia, Austria, Bosnia and Herzegovina, Romania, Switzerland, Serbia, Montenegro, Slovakia, Portugal]"
6,314.0,[United Kingdom]
7,NaN,"[Italy, Czech Republic, Romania, Armenia, Cyprus, France, Ukraine, Azerbaijan]"
8,477.0,[Portugal]


# Temporal: 7 wonders code

In [ ]:
seven_wonders = ['Cairo', 'Rome', 'Beijing', 'Merida', 'Cuzco', 'Lucknow', 'Rio De Janeiro']
pairs = list(itertools.permutations(seven_wonders,2))

pair_df = pd.DataFrame(columns=['src','dst','middle','distance'])
print(pair_df)

for i in pairs:
    query = f"""
    MATCH (a:Airport {{Label:'{i[0]}'}}), (b:Airport {{Label:'{i[1]}'}})

    call gds.shortestPath.dijkstra.stream('Directed_Airport', {{
        sourceNode: a,
        targetNode: b,
        relationshipWeightProperty: 'distance'
    }})
    YIELD sourceNode, targetNode, totalCost, nodeIds
    RETURN gds.util.asNode(sourceNode).Label AS src, gds.util.asNode(targetNode).Label AS dst, nodeIds AS middle, totalCost AS distance;
    """

    answer = gds.run_cypher(query)
    pair_df = pd.concat([pair_df,answer], sort=False)

pair_df_sorted = pair_df.sort_values(by='distance')
df_final = pair_df_sorted.drop_duplicates(subset=['src','dst'], keep='first')
df_final

In [ ]:
permuts = list(itertools.permutations(seven_wonders))

lookup = {(row.src, row.dst): row.distance for row in df_final.itertuples(index=False)}
print(lookup)

distances = pd.DataFrame(columns=['order','distance'])

for i in permuts:
    pre_distance = 0
    
    for x in range(6):
        par = (i[x], i[x+1])
        dist = lookup.get(par)
        pre_distance += dist

    temp_df = pd.DataFrame({
        "order": [i],
        "distance": pre_distance

    })
    distances = pd.concat([distances,temp_df], sort=False)

distances

In [18]:
db.custom_query('CALL gds.graph.list()')

,degreeDistribution,graphName,database,databaseLocation,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema,schemaWithOrientation
0,"{'min': 0, 'max': 145, 'p90': 58, 'p999': 145, 'p99': 123, 'p50': 6, 'p75': 25, 'p95': 78, 'mean': 19.43737574552684}",name,neo4j,local,4298 KiB,4401368,503,9777,"{'readConcurrency': 4, 'undirectedRelationshipTypes': [], 'jobId': 'cdf70914-4f53-4802-b48b-a97a82456c51', 'logProgress': True, 'query': 'MATCH (a:Airport {continent:'Europe'})-[:TO]->(a2:Airport {continent:'Europe'}) WITH gds.graph.project('name',a,a2) AS g RETURN g.graphName, g.nodeCount, g.relationshipCount;', 'inverseIndexedRelationshipTypes': []}",0.03872,2025-07-14T21:32:07.601287900-06:00,2025-07-14T21:32:07.601287900-06:00,"{'graphProperties': {}, 'nodes': {'__ALL__': {}}, 'relationships': {'__ALL__': {}}}","{'graphProperties': {}, 'nodes': {'__ALL__': {}}, 'relationships': {'__ALL__': {'direction': 'DIRECTED', 'properties': {}}}}"
